In diesem Notebook werden auf verschiedenen Modalitäten verschiedene Modelle trainiert, angewendet und ausgewertet. Jeweils: Random Forest, XGBoost, Logistic Regression, SVM, Neuronales Netz

In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Vorbereitung: Data Split

In [14]:
df = pd.read_csv('features_enriched_tab_img_text.csv')
print(f"Orginal Dataset shape: {df.shape}")

train, test = train_test_split(df, test_size=0.2, stratify=df['label'], random_state=2)
print(train.shape)
print(test.shape)
# store them to csv
train.to_csv('train.csv', index=False)
test.to_csv('test.csv', index=False)



train_X = train.drop('label', axis=1)
train_y = train['label']

test_X = test.drop('label', axis=1)
test_y = test['label']
print(train_X.shape, train_y.shape, test_X.shape, test_y.shape)

# save ti 

Orginal Dataset shape: (1410, 562)
(1128, 562)
(282, 562)
(1128, 561) (1128,) (282, 561) (282,)


# Text

# Tabular

# Image

# Multimodal